<a href="https://colab.research.google.com/github/KunalSutar/CNN---Image-Classifier-/blob/main/Flant_T5_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers datasets peft

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Loading and Preprocessing

In [ ]:
from datasets import load_dataset

data = load_dataset("Fredithefish/Instruction-Tuning-with-GPT-4-RedPajama-Chat")
data

In [ ]:
df = data["train"].to_pandas()
df

,text
0,<human>: Give three tips for staying healthy.\...
1,<human>: What are the three primary colors?\n<...
2,<human>: Describe the structure of an atom.\n<...
3,<human>: How can we reduce air pollution?\n<bo...
4,<human>: Describe a time when you had to make ...
...,...
51997,<human>: Generate an example of what a resume ...
51998,<human>: Arrange the items given below in the ...
51999,<human>: Write an introductory paragraph about...
52000,<human>: Generate a list of five things one sh...


In [ ]:
df["text"][0]

'<human>: Give three tips for staying healthy.\n<bot>: 1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.'

In [ ]:
df["Human"] = df["text"].apply(lambda x:x.split("<bot>:")[0].replace("<human>:","Human : ")+ "Assistant :")
df["Assistant"] = df["text"].apply(lambda x:x.split("<bot>:")[-1])

df

,text,Human,Assistant
0,<human>: Give three tips for staying healthy.\...,Human : Give three tips for staying healthy.\...,1. Eat a balanced and nutritious diet: Make s...
1,<human>: What are the three primary colors?\n<...,Human : What are the three primary colors?\nA...,"The three primary colors are red, blue, and y..."
2,<human>: Describe the structure of an atom.\n<...,Human : Describe the structure of an atom.\nA...,An atom is the basic building block of all ma...
3,<human>: How can we reduce air pollution?\n<bo...,Human : How can we reduce air pollution?\nAss...,There are several ways to reduce air pollutio...
4,<human>: Describe a time when you had to make ...,Human : Describe a time when you had to make ...,"As an AI assistant, I do not have my own pers..."
...,...,...,...
51997,<human>: Generate an example of what a resume ...,Human : Generate an example of what a resume ...,**John Doe**\n\n**Contact Information:**\n\n1...
51998,<human>: Arrange the items given below in the ...,Human : Arrange the items given below in the ...,Me eating cake.
51999,<human>: Write an introductory paragraph about...,Human : Write an introductory paragraph about...,"Michelle Obama, born January 17, 1964, in Chi..."
52000,<human>: Generate a list of five things one sh...,Human : Generate a list of five things one sh...,1. Evaluate your reasons: It's important to t...


In [ ]:
df["Human"][0]

'Human :  Give three tips for staying healthy.\nAssistant :'

In [ ]:
df["Assistant"][0]

' 1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.'

In [ ]:
df = df[["Human","Assistant"]]
df

,Human,Assistant
0,Human : Give three tips for staying healthy.\...,1. Eat a balanced and nutritious diet: Make s...
1,Human : What are the three primary colors?\nA...,"The three primary colors are red, blue, and y..."
2,Human : Describe the structure of an atom.\nA...,An atom is the basic building block of all ma...
3,Human : How can we reduce air pollution?\nAss...,There are several ways to reduce air pollutio...
4,Human : Describe a time when you had to make ...,"As an AI assistant, I do not have my own pers..."
...,...,...
51997,Human : Generate an example of what a resume ...,**John Doe**\n\n**Contact Information:**\n\n1...
51998,Human : Arrange the items given below in the ...,Me eating cake.
51999,Human : Write an introductory paragraph about...,"Michelle Obama, born January 17, 1964, in Chi..."
52000,Human : Generate a list of five things one sh...,1. Evaluate your reasons: It's important to t...


In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(
    df,
    test_size=0.2,
    shuffle=True,
    random_state=0
)

train_data

,Human,Assistant
34452,Human : Give three reasons why long-term plan...,1) Anticipating and Preempting Future Challen...
11969,Human : Sort the following terms based on the...,"Helium, Lithium, Uranium."
26533,Human : Identify the appropriate example of a...,"""He eats breakfast every morning."" In this se..."
41946,Human : Copy the following text while making ...,Jane is a driven and resolute student who con...
44683,Human : Compute the following: 2+3\nAssistant :,The result of the computation is 5.
...,...,...
21243,Human : Perform a Google search and find a we...,"After conducting a Google search, I found a w..."
45891,Human : Compose five sentences using the verb...,1. John sat quietly in the garden as he conte...
42613,Human : Emphasize one word in this sentence\n...,He *just* wanted to be heard.
43567,Human : What year did the Great Recession sta...,The Great Recession started in the United Sta...


In [ ]:
train_data.to_parquet("/content/drive/MyDrive/FlantT5Chatbot/train.parquet",index=False)
test_data.to_parquet("/content/drive/MyDrive/FlantT5Chatbot/test.parquet",index=False)

# Chatbot

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model, TaskType
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

In [ ]:
train_df = pd.read_parquet("/content/drive/MyDrive/FlantT5Chatbot/train.parquet")
test_df = pd.read_parquet("/content/drive/MyDrive/FlantT5Chatbot/test.parquet")

Converting to Huggingface dataset

In [ ]:
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

train_data

Dataset({
    features: ['Human', 'Assistant'],
    num_rows: 41601
})

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def preprocess_function(sample,padding="max_length"):
    model_inputs = tokenizer(sample["Human"], max_length=256, padding=padding, truncation=True)
    labels = tokenizer(sample["Assistant"], max_length=256, padding=padding, truncation=True)
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train_tokenized_dataset = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
test_tokenized_dataset = test_data.map(preprocess_function, batched=True, remove_columns=test_data.column_names)
print(f"Keys of tokenized dataset: {list(train_tokenized_dataset.features)}")

In [ ]:
lora_congif = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q","v"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [ ]:
model = get_peft_model(model,lora_config)
model.print_trainable_parameters()

In [ ]:
label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from hugginface_hub import notebook_login
notebook_login()

In [ ]:
output_dir="lora-flan-t5-large"
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    learning_rate=1e-3,
    num_train_epochs=1,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="epoch",
    save_strategy="epoch",
    report_to="tensorboard",
    push_to_hub = True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_tokenized_dataset,
)

In [ ]:
model.config.use_cache = False

In [ ]:
trainer.train()

peft_save_model_id="lora-flan-t5-large"
trainer.model.save_pretrained(peft_save_model_id, push_to_hub=True)
tokenizer.save_pretrained(peft_save_model_id, push_to_hub=True)
trainer.model.base_model.save_pretrained(peft_save_model_id, push_to_hub=True)